In [2]:
from datetime import datetime, timedelta

import pandas as pd
import numpy as np 
from pandas import DataFrame
import nltk
nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer

Opralog['Date']=pd.to_datetime(Opralog.ENTRY_TIMESTAMP)

Opralog=Opralog[Opralog['WELLNAME'].isin(['A01','D10','D13','D14Z','D15','D19','D21','D28','D29','D35','Islay','N05','N12','N37z','N52Y','N53',
                                                'N54Z','N56z','NGA','NGBST'])]



# Opralog_pwri=Opralog[Opralog['SHADOW_COMMENT'].str.contains('PWRI', na=False)] 
# Opralog_pwri=Opralog[Opralog['SHADOW_COMMENT'].str.contains('C104', na=False)] 

# Opralog_pwri=Opralog[Opralog['SHADOW_COMMENT'].str.contains('stim', na=False)] 

[nltk_data] Downloading package punkt to /home/l0533643/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
import nltk.classify.util
from collections import Counter
from nltk import ngrams
import numpy as np
import pandas as pd
import re
import string
string.punctuation
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

stopwords.extend(["well", "D14","D15","N55","D19","N54","D10","D13","D29","D28","D21","N56","D35","N12","N53","N05",
"A01","N37","N51","N52","ISLAY","NGBST","NGA","N50","bar","N56","also","however"])
exception = {'CO2','C02'}


from stemming.porter2 import stem

#remove words that are in NLTK stopwords list
not_stopwords = {'no', 'don', 'not'}
new_stopwords = set([word for word in stopwords if word not in not_stopwords])

from nltk.stem.wordnet import WordNetLemmatizer

ps = nltk.PorterStemmer()
lm =  WordNetLemmatizer()
number='0123456789'

def no_space(sentence):
    sentence = [x.replace(' ', '') for x in sentence]

    return sentence

#initial clean removes punctuation and sets all text to lower case
def initial_clean(text):
   
    if text not in exception:
        text = re.sub('(N)(\d+)','',str(text))   # remove wells and equipments names

    #remove punctuation
    text = "".join([char for char in text if char not in string.punctuation])
    text = re.sub("(\')",'',str(text))
    text = re.sub(r"(\W)",r' ',str(text))
    text = re.sub(r" +",r' ',str(text))
    text = re.sub("\/"," ",str(text))
    text = re.sub(r'\b\d+(?:\.\d+)?\s+','',str(text))
    text = re.sub('^[0-9 ]+', '', str(text))
    text = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ",str(text))
    nopunct = text.lower()
    nonumb= "".join([char for char in nopunct if char not in number])
   
    return nonumb

import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
nlp = spacy.load('en', disable = ['ner', 'textcat', 'similarity', 'merge_noun_chunks', 'merge_entities', 'tensorizer', 'parser', 'sbd', 'sentencizer'])


def lemmatize_sentences(nlp_spacy, text_series):
    lemmatized_sentences = []
    for text in text_series.astype(str):
        lemmas = []
        for token in nlp_spacy(text):
            lemmas.append(token.lemma_)
        lemmatized_sentences.append(' '.join(lemmas))
    return lemmatized_sentences

#stems words, applies the spelling corrector and removes any stopwords
def clean_data_stem(text):
    #split review into words tokenized
    tokens = re.split('\W+',text) 
    #text = [ ps.stem(correction(word)) for word in tokens if word not in stopwords]
    text = [ word for word in tokens if word not in stopwords]
    return text

def data_for_ngram(text):
    new_text=""
    for word in text: #.split():
        if word not in stopwords:
            #new_text=new_text+" "+ps.stem(correction(word))
            #new_text=new_text+" "+ps.stem(word)
            new_text=new_text+" "+word
            
    return new_text

def ngram_count(words, text):
     
    count = 0
    for word in text:  
        if word == words:
            count +=1 
    return count


def ngram_list(text):

    list_ngram=[]
    bigrams=ngrams(text.split(),2)
    for gram in bigrams:
        newgram=""
        for word in gram:
            newgram=newgram+word+" "
        list_ngram.append(newgram[0:len(newgram)-1])
    #trigrams=ngrams(text.split(),3)
    #for gram in trigrams:
    #    newgram=""
    #    for word in gram:
    #        newgram=newgram+word+" "
    #    list_ngram.append(newgram[0:len(newgram)-1])

    return list_ngram



def feature_selection(data):

    #does the initial clean ready to find any double bonded words
    data['Initial Clean']=data['SHADOW_COMMENT'].apply(lambda x: initial_clean(x))
    data['Stem']=data['Initial Clean'].apply(lambda x: clean_data_stem(x))
    data['ngram']=data['Initial Clean'].apply(lambda x: data_for_ngram(x))
    data['grams']=data['ngram'].apply(lambda x: ngram_list(x))
    
    return data


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/l0533643/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
Opralog['clean_opra']=lemmatize_sentences(nlp,Opralog['clean'])


In [ ]:
Opralog['clean_opra']=Opralog.clean_opra.apply(lambda x: clean_data_stem(x))


In [ ]:
Opralog['ngram']=Opralog.clean_opra.apply(lambda x: data_for_ngram(x))
Opralog=Opralog.loc[~Opralog['ngram'].str.contains('nan')]
Opralog['grams']=Opralog.ngram.apply(lambda x: ngram_list(x))

In [ ]:
Opralog['ngrams2']= Opralog.grams.apply(lambda x:  data_for_ngram(no_space(x)))


In [ ]:

Opralog_stim=Opralog[Opralog['ngram'].str.contains('stim')]  

Opralog_stim #140 times

# On the topic of stimming;
# •	Topic 3 looked like the best list of related words for stimming Dunbar wells 
# (3, '0.008*"drain stimulation" + 0.008*"drain stim" + 0.008*"stimulation try" + 0.008*"route drain"')
# •	No distinct topic looks best suited to Alwyn wells. Would maybe expect “C104” or “test” to be relevant terms.

# For changes to a wells flowing behaviour (not necessarily a start or shut-in);
# •	All wells- Routed, Rerouted, swung, choke, choked, cut back, test, lined up to 
# •	Dunbar wells – Suction pressure
# •	Dunbar & Islay – HP Mode, LP Mode
# •	A01 – BDV bypass, arrivals, PWRI

# For wells about to be shut-in/started;
# •	CITHP, DHSV, equalise, equalised, remove gas cap, offload gas cap

Opralog_c104=Opralog[Opralog['SHADOW_COMMENT'].str.contains('C104')]
Opralog_c104 #82 times

Opralog.LOGBOOK_NAME.value_counts()


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

df = []
df = pd.DataFrame(columns=['explained_variance_ratio', 'components'])
j=0
                      
for i in range(100, 1500, 100):
    tfidf_vec = TfidfVectorizer(use_idf=True, norm='l2')
    svd = TruncatedSVD(n_components=i)
    transformed_x_train = tfidf_vec.fit_transform(Opralog["ngram"])
    x_train_svd = svd.fit_transform(transformed_x_train)
    df.loc[j,'explained_variance_ratio'] = svd.explained_variance_ratio_.sum()
    df.loc[j,'components'] = i
    j=j+1

 

In [ ]:
df
import numpy as np
import pandas as pd
from IPython.display import display
from tqdm import tqdm
from collections import Counter
import ast

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sb

from sklearn.feature_extraction.text import CountVectorizer
#from textblob import TextBlob
import scipy.stats as stats

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE

from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
output_notebook()

%matplotlib inline

In [ ]:
tfidf_vec = TfidfVectorizer(use_idf=True, norm='l2')
svd = TruncatedSVD(n_components=350)
transformed_x_train = tfidf_vec.fit_transform(Opralog["ngram"])
lsa_topic_matrix= svd.fit_transform(transformed_x_train)
    
lsa_keys = get_keys(lsa_topic_matrix)
lsa_categories, lsa_counts = keys_to_counts(lsa_keys)

top_n_words_lsa = get_top_n_words(10, lsa_keys, small_document_term_matrix, small_count_vectorizer)

for i in range(len(top_n_words_lsa)):
    print("Topic {}: ".format(i+1), top_n_words_lsa[i])

In [ ]:
import gzip
import gensim
import logging
import os

# read the tokenized reviews into a list
    # each review item becomes a serries of words
    # so this becomes a list of lists
documents = list(Opralog.clean_opra)
    
    # build vocabulary and train model
model = gensim.models.Word2Vec(
        documents,
        size=150,
        window=10,
        min_count=2,
        workers=10)
model.train(documents, total_examples=len(documents), epochs=10)

documents
        
    
redundant_terms = [ 'baker drawer', 'drawer', 'a annulus', 'b annulus', 'c annulus', 'annulus a', 'annulus b', 'annulus c',
                   'a annuli', 'b annuli', 'c annuli', 'annuli a', 'annuli b', 'annuli c', 'annuli', 'alarm', 'high alarm',
                   'annulus', 'Words','bleed off', 'top up', ' a ', ' b ', ' c ',   'bleed', 'update', 'bled', 'bleeding', 'thermal', 'corrosion coupon',
                   'wims', 'mls/min', 'injection', 'inhibitor', 'leak off', 'leak rate', 'gas lift', 'scale pump']
    


In [ ]:
w1 = ['stim']
print("Most similar to {0}".format(w1),model.wv.most_similar(positive=w1,topn=10))   


In [ ]:
# Top words on unknown
list_wells=Opralog.WELLNAME.unique()
#list_years=Opralog.Year.unique()
list_wells=list_wells.tolist()
vec = CountVectorizer()


In [ ]:
for i in list_years:
    df2 = pd.DataFrame(vec.fit_transform(Opralog.loc[Opralog.Year==i,"ngram"]).toarray(), 
                      columns=vec.get_feature_names())
    print(i)
    print(df2.sum().sort_values(ascending=False).head(20)) 
    

In [ ]:
for i in list_wells:
    df2 = pd.DataFrame(vec.fit_transform(Opralog.loc[Opralog.WELLNAME==i,"ngram"]).toarray(), 
                      columns=vec.get_feature_names())
    print(i)
    print(df2.sum().sort_values(ascending=False).head(20)) 
    

In [ ]:
df = pd.DataFrame(vec.fit_transform(Opralog["ngram"]).toarray(), 
                      columns=vec.get_feature_names())

print(df.sum().sort_values(ascending=False).head(40))

In [ ]:
vec = CountVectorizer()
df2 = pd.DataFrame(vec.fit_transform(Opralog["ngrams2"]).toarray(), 
                  columns=vec.get_feature_names())
print(df2.sum().sort_values(ascending=False).head(40))

In [ ]:
Opralog_Dunbar=Opralog.loc[Opralog.LOGBOOK_NAME.isin(['Dunbar CCR Logbook','Dunbar Welltech Logbook'])]
Opralog_Alwyn=Opralog.loc[Opralog.LOGBOOK_NAME.isin(['Alwyn North CCR Logbook','NAA Operators Logbook'])]

In [ ]:
import random
import word2vec
from gensim import corpora
import pickle
#text_data = []
#with open('dataset.csv') as f:
#    for line in f:
#        tokens = prepare_text_for_lda(line)
#        if random.random() > .99:
#            print(tokens)
#            text_data.append(tokens)
           
    #clean_opra
    #grams

dictionary = corpora.Dictionary(Opralog_Dunbar["grams"])
corpus = [dictionary.doc2bow(text) for text in Opralog_Dunbar["grams"]]

dictionary_alwyn = corpora.Dictionary(Opralog_Alwyn["grams"])
corpus_alwyn = [dictionary_alwyn.doc2bow(text) for text in Opralog_Alwyn["grams"]]


pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

pickle.dump(corpus_alwyn, open('corpus_alwyn.pkl', 'wb'))
dictionary_alwyn.save('dictionary_alwyn.gensim')

# DUNBAR TOPIC MODELLING

In [ ]:
import gensim
NUM_TOPICS =6
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('topic_model_Dunbar.gensim')
topics = ldamodel.print_topics(num_words=3)
for topic in topics:
    print(topic)

In [ ]:
import pyLDAvis.gensim

dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('topic_model_Dunbar.gensim')

lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)           
            

# ALWYN TOPIC MODELLING

In [ ]:
import gensim
NUM_TOPICS = 6
ldamodel = gensim.models.ldamodel.LdaModel(corpus_alwyn, num_topics = NUM_TOPICS, id2word=dictionary_alwyn, passes=15)
ldamodel.save('topic_model_Alwyn.gensim')
topics = ldamodel.print_topics(num_words=3)
for topic in topics:
    print(topic)
   

In [ ]:
 
import pyLDAvis.gensim

dictionary = gensim.corpora.Dictionary.load('dictionary_alwyn.gensim')
corpus = pickle.load(open('corpus_alwyn.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('topic_model_Alwyn.gensim')

lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)      

#APPLY TOPIC MODELLING TO NEW DATA


In [ ]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

documents=Opralog.grams

#for top in lda.print_topics():
#    print(top
          
print(lda) # LdaModel(num_terms=2173, num_topics=5, decay=0.5, chunksize=2000)
lda[corpus] #<gensim.interfaces.TransformedCorpus at 0x7fbda79a05c0>
enumerate(lda_corpus)
documents[2]
lda_corpus

In [ ]:
# https://stackoverflow.com/questions/20984841/topic-distribution-how-do-we-see-which-document-belong-to-which-topic-after-doi
corpus

In [ ]:
## Libraries to download
import itertools
from itertools import chain
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

# Assinging the topics to the document in corpus
lda_corpus = ldamodel[corpus] # <gensim.interfaces.TransformedCorpus at 0x7fbda79a06d8>

# Find the threshold, let's set the threshold to be 1/#clusters,
# To prove that the threshold is sane, we average the sum of all probabilities:
scores = list(chain(*[[score for topic_id,score in topic] \
                     for topic in [doc for doc in lda_corpus]]))

threshold = sum(scores)/len(scores)
print(threshold)

cluster1 = [j for i,j in zip(lda_corpus,documents) if i[0][1] > threshold]
cluster2 = [j for i,j in zip(lda_corpus,documents) if i[1][1] > threshold]
cluster3 = [j for i,j in zip(lda_corpus,documents) if i[2][1] > threshold]

print(cluster3)